# Coursera Capstone
## The Battle of the Neighborhoods
This document will be used for the Coursera Capstone project, part of the IBM Data Science Professional Certificate.

In [55]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

# All requested packages already installed.



### Create a dataframe of neighborhoods in Minneapolis
As far as I could tell, the data for Minneapolis neighborhoods did not come attached with latitude and longitude values. The following section creates a dataframe with neighborhood names and attached lat/long values.

In [4]:
# Data retrieved from
# https://opendata.minneapolismn.gov/datasets/minneapolis-neighborhoods
# Data downloaded and placed in local directory

with open('Minneapolis_Neighborhoods.geojson.json') as jsondata:
    mpls_data = json.load(jsondata)

Define `mpls_nbhds` to be the features of the geojson file imported above.

In [6]:
mpls_nbhds = mpls_data['features']

In [7]:
# Look at the first entry. Note the neighborhood name has key 'BDNAME'.
mpls_nbhds[0]

{'type': 'Feature',
 'properties': {'FID': 1,
  'BDNAME': 'Phillips West',
  'BDNUM': '90',
  'TEXT_NBR': '90',
  'Shape_STAr': 10669249.7418,
  'Shape_STLe': 14403.8859339,
  'NCR_LINK': 'http://www.nrp.org/r2/Neighborhoods/Orgs/PHW.html',
  'IMAGE': 'PHW',
  'SHAPE_Length': 0.04580106358679052,
  'SHAPE_Area': 0.00011302161937450821},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-93.26258075899995, 44.96090821400003],
     [-93.26258068799996, 44.960896674000026],
     [-93.26257007799995, 44.95912773800006],
     [-93.26257006199995, 44.959125131000064],
     [-93.26257002699998, 44.95911920300006],
     [-93.26257002299997, 44.95911835000004],
     [-93.26257002199998, 44.95911830400007],
     [-93.26257006099996, 44.95909969500008],
     [-93.26257174699998, 44.95830091700003],
     [-93.26257379799995, 44.95732838600003],
     [-93.26257385099996, 44.95732643300005],
     [-93.26257396599999, 44.957322271000066],
     [-93.26257794099996, 44.95717704200007],
     [-

In [39]:
# Initalize the dataframe
column_names = ['Neighborhood','Latitude', 'Longitude']
nbhds = pd.DataFrame(columns=column_names)
nhbds

In [41]:
# Fill in the neighborhood names. The latitude and longitude are filled in the next step.
for data in mpls_nbhds:
    nbhd_name = data['properties']['BDNAME']
    nbhds = nbhds.append({'Neighborhood':nbhd_name}, ignore_index=True)

In [51]:
nbhds.head()

,Neighborhood,Latitude,Longitude
0,Phillips West,1,NaN
1,Downtown West,NaN,NaN
2,Downtown East,NaN,NaN
3,Ventura Village,NaN,NaN
4,Sumner - Glenwood,NaN,NaN


#### Populate the latitude and longitude using Geopy

In [90]:
counter = 0
missing_addresses = []
for nbhd in nbhds['Neighborhood']:
    address = nbhd + ', Minneapolis, MN'
    geolocator = Nominatim(user_agent="mpls_explorer")
    location = geolocator.geocode(address)
    if location is None:
        print(address+' is not found on geopy.')
        missing_addresses = missing_addresses +[nbhd]
    else:
        latitude = location.latitude
        longitude = location.longitude
        nbhds['Latitude'][counter] = latitude
        nbhds['Longitude'][counter] = longitude
    counter+=1
print(missing_addresses)
nbhds.head()

Downtown West, Minneapolis, MN is not found on geopy.
Ventura Village, Minneapolis, MN is not found on geopy.
Humboldt Industrial Area, Minneapolis, MN is not found on geopy.
South Uptown, Minneapolis, MN is not found on geopy.
Mid - City Industrial, Minneapolis, MN is not found on geopy.
Nicollet Island - East Bank, Minneapolis, MN is not found on geopy.
['Downtown West', 'Ventura Village', 'Humboldt Industrial Area', 'South Uptown', 'Mid - City Industrial', 'Nicollet Island - East Bank']


,Neighborhood,Latitude,Longitude
0,Phillips West,44.9539,-93.2663
1,Downtown West,NaN,NaN
2,Downtown East,44.975,-93.2599
3,Ventura Village,NaN,NaN
4,Sumner - Glenwood,44.9837,-93.2914


There are 6 neighborhoods that Geopy could not identify. We can manually input these by pulling the location data from a google search.

In [92]:
Downtown_West =[44.9742, -93.2733]
Ventura_Village = [44.9618, -93.2582]
Humboldt_Industrial_Area = [45.0421, -93.3077]
South_Uptown = [44.9411, -93.2911]
Mid_City_Industrial = [44.9989, -93.2178]
Nicollet_Island = [44.9879, -93.2629]

missing_latlon = [Downtown_West, Ventura_Village, Humboldt_Industrial_Area, South_Uptown, Mid_City_Industrial, Nicollet_Island]

In [96]:
missing_dict={}
for j in range(0,6):
    entry ={missing_addresses[j]:missing_latlon[j]}
    missing_dict.update(entry)
missing_dict

{'Downtown West': [44.9742, -93.2733],
 'Ventura Village': [44.9618, -93.2582],
 'Humboldt Industrial Area': [45.0421, -93.3077],
 'South Uptown': [44.9411, -93.2911],
 'Mid - City Industrial': [44.9989, -93.2178],
 'Nicollet Island - East Bank': [44.9879, -93.2629]}

In [112]:
# Fill in missing lat/long values into our dataframe
for j in range(0,6):
    value=nbhds.index[nbhds['Neighborhood']==missing_addresses[j]]
    nbhds.loc[value[0], 'Latitude'] = missing_dict[missing_addresses[j]][0]
    nbhds.loc[value[0], 'Longitude'] = missing_dict[missing_addresses[j]][1]

nbhds.head()

,Neighborhood,Latitude,Longitude
0,Phillips West,44.9539,-93.2663
1,Downtown West,44.9742,-93.2733
2,Downtown East,44.975,-93.2599
3,Ventura Village,44.9618,-93.2582
4,Sumner - Glenwood,44.9837,-93.2914


In [114]:
nbhds.shape

(87, 3)

According to Wikipedia, there are 81 official neighborhoods. Here we tally 87. Some cross-referencing perhaps is in order.

### Use FourSquare API to gather data about venues in the neighborhoods.
#### L

In [158]:
# Define FourSquare credentials
CLIENT_ID = 'JSN5MX1DKF5XI3CXVZADJMU5LZE5FMLT2COF00LRJDFMFWIK' # your Foursquare ID
CLIENT_SECRET = 'ZST1WYPJCG2J2LGQGUER23BPAC1OMF1BKYC4WQKSSRD3WC1T' # your Foursquare Secret
VERSION = '20200604' # Foursquare API version

search_query ='Brewery'

address = 'Minneapolis, MN'
geolocator = Nominatim(user_agent="mpls_explorer")
location = geolocator.geocode(address)
mpls_latitude = location.latitude # neighborhood latitude value
mpls_longitude = location.longitude # neighborhood longitude value

LIMIT = 200
radius = 25000
url = 'https://api.foursquare.com/v2/venues/\
search?client_id={}&client_secret={}\
&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, mpls_latitude, mpls_longitude, VERSION, search_query, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/search?client_id=JSN5MX1DKF5XI3CXVZADJMU5LZE5FMLT2COF00LRJDFMFWIK&client_secret=ZST1WYPJCG2J2LGQGUER23BPAC1OMF1BKYC4WQKSSRD3WC1T&ll=44.9772995,-93.2654692&v=20200604&query=Brewery&radius=25000&limit=200'

In [159]:
results = requests.get(url).json()

In [160]:
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

/Users/no50free/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,41326e00f964a52060171fe3,Minneapolis Town Hall Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,1430 Washington Ave S,3rd St S,44.973320,-93.247628,"[{'label': 'display', 'lat': 44.97332009749321...",...,United States,"[1430 Washington Ave S (3rd St S), Minneapolis...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55eb2fee498e99d010fbd68b,twincities brewery tours,"[{'id': '4bf58dd8d48988d1f6931735', 'name': 'G...",v-1594759764,False,NaN,NaN,44.978428,-93.265959,"[{'label': 'display', 'lat': 44.978428, 'lng':...",...,United States,"[Minneapolis, MN, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50f3286ce4b0c1f511605e93,Banjo Kid's Downtown Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,521 S 7th St,NaN,44.973312,-93.264725,"[{'label': 'display', 'lat': 44.97331237792969...",...,United States,"[521 S 7th St, Minneapolis, MN 55415, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41326e00f964a52008171fe3,The Herkimer Pub & Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,2922 Lyndale Avenue South,btwn W 29th & Lake,44.949214,-93.288244,"[{'label': 'display', 'lat': 44.94921355801643...",...,United States,[2922 Lyndale Avenue South (btwn W 29th & Lake...,40298890,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52e6ad8c498efd5a184d1105,LynLake Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,2934 Lyndale Ave S,W Lake Street,44.948919,-93.288345,"[{'label': 'display', 'lat': 44.94891907707041...",...,United States,"[2934 Lyndale Ave S (W Lake Street), Minneapol...",87032157,"Lyn Lake, Minneapolis, MN",NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
df.shape

(50, 25)

In [162]:
df

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,41326e00f964a52060171fe3,Minneapolis Town Hall Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,1430 Washington Ave S,3rd St S,44.973320,-93.247628,"[{'label': 'display', 'lat': 44.97332009749321...",...,United States,"[1430 Washington Ave S (3rd St S), Minneapolis...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55eb2fee498e99d010fbd68b,twincities brewery tours,"[{'id': '4bf58dd8d48988d1f6931735', 'name': 'G...",v-1594759764,False,NaN,NaN,44.978428,-93.265959,"[{'label': 'display', 'lat': 44.978428, 'lng':...",...,United States,"[Minneapolis, MN, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50f3286ce4b0c1f511605e93,Banjo Kid's Downtown Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,521 S 7th St,NaN,44.973312,-93.264725,"[{'label': 'display', 'lat': 44.97331237792969...",...,United States,"[521 S 7th St, Minneapolis, MN 55415, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41326e00f964a52008171fe3,The Herkimer Pub & Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,2922 Lyndale Avenue South,btwn W 29th & Lake,44.949214,-93.288244,"[{'label': 'display', 'lat': 44.94921355801643...",...,United States,[2922 Lyndale Avenue South (btwn W 29th & Lake...,40298890,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52e6ad8c498efd5a184d1105,LynLake Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,2934 Lyndale Ave S,W Lake Street,44.948919,-93.288345,"[{'label': 'display', 'lat': 44.94891907707041...",...,United States,"[2934 Lyndale Ave S (W Lake Street), Minneapol...",87032157,"Lyn Lake, Minneapolis, MN",NaN,NaN,NaN,NaN,NaN,NaN
5,4ea0d6c400394cc284a862c4,Franklin Terrace Brewery Redux,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,4412 Columbus Ave,NaN,44.922766,-93.264196,"[{'label': 'display', 'lat': 44.92276599999999...",...,United States,"[4412 Columbus Ave, Minneapolis, MN 55407, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4de30667ae60e7f3abfeadd7,Black & Tan Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,2227 Monroe St NE,NaN,45.010180,-93.252054,"[{'label': 'display', 'lat': 45.01018, 'lng': ...",...,United States,"[2227 Monroe St NE, Minneapolis, MN 55418, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,45bf30d0f964a5201d421fe3,Granite City Food and Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1594759764,False,851 Rosedale Ctr,NaN,45.012886,-93.169644,"[{'label': 'display', 'lat': 45.01288558042535...",...,United States,"[851 Rosedale Ctr, Roseville, MN 55113, United...",NaN,NaN,2107537,https://www.grubhub.com/restaurant/granite-cit...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
8,53953849498e574c69cbeaa0,Bitter Minnesota Brewery Tours,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1594759764,False,345 N 6th Ave,NaN,44.985346,-93.277626,"[{'label': 'display', 'lat': 44.985346, 'lng':...",...,United States,"[345 N 6th Ave, Minneapolis, MN 55401, United ...",87891475,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,507753e6e4b0468862fa41ed,Barrio Truck @ Fulton Brewery,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1594759764,False,414 6th Ave N,NaN,44.984942,-93.278887,"[{'label': 'display', 'lat': 44.984942, 'lng':...",...,United States,"[414 6th Ave N, Minneapolis, MN 55401, United ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
